In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.head()

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [4]:
test.head()

,Image
0,508.jpg
1,246.jpg
2,473.jpg
3,485.jpg
4,128.jpg


In [5]:
print(train['target'].unique())

['manipuri' 'bharatanatyam' 'odissi' 'kathakali' 'kathak' 'sattriya'
 'kuchipudi' 'mohiniyattam']


In [6]:
Class_map={'manipuri':0, 'bharatanatyam':1, 'odissi':2 ,'kathakali':3, 'kathak':4, 'sattriya':5,
 'kuchipudi':6, 'mohiniyattam':7}
inverse_map={0:'manipuri', 1:'bharatanatyam', 2:'odissi' ,3:'kathakali',4: 'kathak', 5:'sattriya',
 6:'kuchipudi', 7:'mohiniyattam'}
train['target']=train['target'].map(Class_map)


In [7]:
train.head()

,Image,target
0,96.jpg,0
1,163.jpg,1
2,450.jpg,2
3,219.jpg,3
4,455.jpg,2


In [8]:
img_h,img_w= (224,224)

In [9]:
train_img=[]
train_label=[]
j=0
path='dataset/train'
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(img_h,img_w))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['target'][j])
    j=j+1

100%|██████████| 364/364 [00:03<00:00, 93.75it/s] 


In [10]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train_img, train_label, test_size=0.30, shuffle= True)

In [11]:
test_img=[]
path='dataset/test'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(img_h,img_w))
    img=img.astype('float32')
    test_img.append(img)

100%|██████████| 156/156 [00:02<00:00, 77.55it/s]


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,# divide each input by its std
        rescale=1./255,
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

test_datagen= ImageDataGenerator(rescale=1./255)
valid_datagen= ImageDataGenerator(rescale=1./255)
train_datagen.fit(x_train)
test_datagen.fit(test_img)
valid_datagen.fit(x_valid)

C:\Users\Manish Sehrawat\anaconda3\envs\TensorFlow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
train_img=np.array(train_img)
x_train= np.array(x_train)
x_valid= np.array(x_valid)
y_train= np.array(y_train)
y_valid= np.array(y_valid)
test_img=np.array(test_img)
train_label=np.array(train_label)
print("Shape of training data=",x_train.shape," and shape of labels of training data= ",y_train.shape)
print("Shape of validation data=",x_valid.shape," and shape of labels of validation data= ",y_valid.shape)
print("Shape of test data=",test_img.shape)

Shape of training data= (254, 224, 224, 3)  and shape of labels of training data=  (254,)
Shape of validation data= (110, 224, 224, 3)  and shape of labels of validation data=  (110,)
Shape of test data= (156, 224, 224, 3)


In [14]:
from tensorflow.keras.applications.vgg16 import VGG16 
from tensorflow.keras.applications.vgg16 import preprocess_input

In [34]:
import seaborn as sns
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Lambda,Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg19 import VGG19 
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.optimizers import RMSprop,Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import image
vggmodel=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3),pooling='max')
vggmodel.trainable=False
model=Sequential([
    vggmodel,
    Dense(units=1024,activation='relu',kernel_initializer='uniform'),
    Dropout(0.25),
    Dense(units=512,activation='relu'),
    Dropout(0.25),
    Dense(units=8,activation='softmax')
])

In [38]:
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
base_model_3=VGG19(include_top=False, weights='imagenet',input_shape=(img_h,img_w,3), pooling='max')

for layer in base_model_3.layers[:-4]:
    layer.trainable=False
#base_model_3.trainable=False
    
model_3=Sequential()
model_3.add(base_model_3)
model_3.add(Flatten())

model_3.add(BatchNormalization())
model_3.add(Dropout(0.2))


model_3.add(Dense(256, activation='relu'))
model_3.add(BatchNormalization())


model_3.add(Dense(64, activation='relu'))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.2))
model_3.add(Dense(8,activation='softmax'))

model_3.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_3.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 512)               20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dense_16 (Dense)             (None, 64)               

In [37]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [39]:
model_3.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=32),epochs=40,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=32),
          verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 8 steps, validate for 4 steps
Epoch 1/40
8/8 [==============================] - 65s 8s/step - loss: 2.3236 - accuracy: 0.2638 - val_loss: 2.6014 - val_accuracy: 0.2909
Epoch 2/40
8/8 [==============================] - 68s 9s/step - loss: 1.6008 - accuracy: 0.5039 - val_loss: 3.5471 - val_accuracy: 0.2727
Epoch 3/40
8/8 [==============================] - 62s 8s/step - loss: 1.3293 - accuracy: 0.5079 - val_loss: 3.9274 - val_accuracy: 0.3727
Epoch 4/40
8/8 [==============================] - 62s 8s/step - loss: 1.0501 - accuracy: 0.6181 - val_loss: 5.3457 - val_accuracy: 0.2455
Epoch 5/40
8/8 [==============================] - 61s 8s/step - loss: 0.9524 - accuracy: 0.6811 - val_loss: 6.4507 - val_accuracy: 0.2545
Epoch 6/40
8/8 [==============================] - 67s 8s/step - loss: 0.8068 - accuracy: 0.7402 - val_loss: 4.8373 - val_accuracy: 0.2909
Epoch 7/40
8/8 [==============================] - 63s 8s/step - loss: 0.7806 - acc

In [42]:
labels = model_3.predict(test_img)
print(labels[:4])
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head(10)
submission.to_csv('submission2.csv', index=False)

[[0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 2.8712414e-08 3.9486597e-35]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 8.0880691e-36 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 2.0442083e-32 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00]]
['odissi', 'mohiniyattam', 'odissi']


In [1]:
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
base_model_3=VGG19(include_top=False, weights='imagenet',input_shape=(img_h,img_w,3), pooling='max')

C:\Users\Manish Sehrawat\anaconda3\envs\TensorFlow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


NameError: name 'img_h' is not defined